# AICROWD AND GOOGLE COLAB

In [1]:
import sys
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path_to_drive = "/content/drive/MyDrive/Colab Notebooks/"
sys.path.append(path_to_drive + 'DevDiabetesLearning/DiabetesLearningAI') #make src files importable

Mounted at /content/drive


In [ ]:
!pip install aicrowd-cli > /dev/null
!aicrowd login

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Please login here: https://api.aicrowd.com/auth/lc3HecHKfMOQHGiBVWbvjTfy_DDfUU8g8nN4UM22F2c
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: www-browser: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links2: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: elinks: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: links: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: lynx: not found
/usr/bin/xdg-open: 851: /usr/bin/xdg-open: w3m: not found
xdg-open: no method available for opening 'https://api.aicrowd.com/auth/lc3HecHKfMOQHGiBVWbvjTfy_DDfUU8g8nN4UM22F2c'
API Key valid
Gitlab access token valid
Saved details success

In [ ]:
# List dataset for this challenge
!aicrowd dataset list -c food-recognition-benchmark-2022


                          Datasets for challenge #962                           
┌───┬────────────────────────────────┬───────────────────────────────┬─────────┐
│ # │ Title                          │ Description                   │    Size │
├───┼────────────────────────────────┼───────────────────────────────┼─────────┤
│ 0 │ random_prediction.json         │ Random prediction for Quick   │ 4.36 MB │
│   │                                │ Submission into Round 2       │         │
│ 1 │ [Round 2]                      │ [Public] Testing Dataset      │    185M │
│   │ public_test_release_2.1.tar.gz │ (contains 2819 images,        │         │
│   │                                │ without annotations)          │         │
│ 2 │ [Round 2]                      │ Training Dataset (contains    │    3.1G │
│   │ public_training_set_release_2… │ 54392 images and 323          │         │
│   │                                │ categories)                   │         │
│ 3 │ [Round 2]             

In [ ]:
# download dataset
# !aicrowd dataset download -c food-recognition-benchmark-2022

In [ ]:
# Manually move downloaded data to drive dir

# Extract tar
aicrowd_dir = path_to_drive + 'DevDiabetesLearning/DiabetesLearningAI/dataset_aiCrowd/'
!cd '/content/drive/MyDrive/Colab Notebooks/DevDiabetesLearning/DiabetesLearningAI/dataset_aiCrowd/test/' && echo "Extracting test dataset" && tar -xvf public_test_release_2.1.tar.gz > /dev/null
!cd '/content/drive/MyDrive/Colab Notebooks/DevDiabetesLearning/DiabetesLearningAI/dataset_aiCrowd/val/' && echo "Extracting val dataset" && tar -xvf public_validation_set_release_2.1.tar.gz > /dev/null
!cd '/content/drive/MyDrive/Colab Notebooks/DevDiabetesLearning/DiabetesLearningAI/dataset_aiCrowd/train/' && echo "Extracting train dataset" && tar -xvf public_training_set_release_2.1.tar.gz > /dev/null

Extracting test dataset
Extracting val dataset
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.quarantine'
tar: Ignoring unknown extended header keyword 'LIBARCHIVE.xattr.com.apple.lastuseddate#PS'
Extracting train dataset


# DIABETES LEARNING



In [2]:
# !pip install matplotlib==3.3.4
# !pip install pycocotools
# !pip install plotly
# !pip install ipyplot
# !pip install opencv-python
# !pip install fastai==2.5.3

     |████████████████████████████████| 11.5 MB 5.4 MB/s 
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 189 kB 5.2 MB/s 
     |████████████████████████████████| 55 kB 2.8 MB/s 
  Attempting uninstall: fastai
    Found existing installation: fastai 1.0.61
    Uninstalling fastai-1.0.61:
      Successfully uninstalled fastai-1.0.61


In [3]:
import sys; print('Python:',sys.version)
import torch; print('Pytorch:',torch.__version__)
import fastai; print('Fastai:',fastai.__version__)

Python: 3.7.12 (default, Jan 15 2022, 18:48:18) 
[GCC 7.5.0]
Pytorch: 1.10.0+cu111
Fastai: 2.5.3


In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.basics import *
from fastai.callback.all import *
from fastai.vision.all import *
import torch
from PIL import Image
from pycocotools.coco import COCO
import cv2
import tqdm

## Data Inspection

In [5]:
# train_folder = "/mnt/c/Users/bruno/Documents/diabetesLearning/foodDatabase/aiCrowd/train/"
# val_folder = "/mnt/c/Users/bruno/Documents/diabetesLearning/foodDatabase/aiCrowd/val/"

train_folder = path_to_drive + 'DevDiabetesLearning/DiabetesLearningAI/dataset_aiCrowd/train/'
val_folder = path_to_drive + 'DevDiabetesLearning/DiabetesLearningAI/dataset_aiCrowd/val/'

In [ ]:
# Fix dataset if necessary

def fix_data(annotations, directiory, VERBOSE = True):
  for n, i in enumerate((annotations['images'])):
   
      img = cv2.imread(directiory+i["file_name"])
 
      if img.shape[0] != i['height']:
          annotations['images'][n]['height'] = img.shape[0]
          if VERBOSE:
            print(i["file_name"])
            print(annotations['images'][n], img.shape)

      if img.shape[1] != i['width']:
          annotations['images'][n]['width'] = img.shape[1]
          if VERBOSE:
            print(i["file_name"])
            print(annotations['images'][n], img.shape)

  return annotations

#fix annotations for training dataset
with open(train_folder + "annotations.json") as f:
  train_annotations_data = json.load(f)
train_annotations_data = fix_data(train_annotations_data, train_folder + "images/")
with open(train_folder + "annotations_fixed.json", 'w') as f:
    json.dump(train_annotations_data, f)

#fix annotations for validation dataset
with open(val_folder + "annotations.json") as f:
  val_annotations_data = json.load(f)
val_annotations_data = fix_data(val_annotations_data, val_folder + "images/")
with open(val_folder + "annotations_fixed.json", 'w') as f:
    json.dump(val_annotations_data, f)

In [6]:
# Define annotation files and dataset to be processed

train_coco = COCO(train_folder + "annotations_fixed.json")
val_coco = COCO(val_folder + "annotations_fixed.json")

coco_dataset = train_coco
coco_dataset_folder = train_folder

loading annotations into memory...
Done (t=6.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.50s)
creating index...
index created!


In [7]:
# Reading all classes

category_ids = coco_dataset.loadCats(coco_dataset.getCatIds())
# category_names = [_["name_readable"] for _ in category_ids]
category_names_dict = { cat["id"] : cat["name"] for cat in category_ids}

category_ids[0]

{'id': 50,
 'name': 'beetroot-steamed-without-addition-of-salt',
 'name_readable': 'Beetroot, steamed, without addition of salt',
 'supercategory': 'food'}

In [8]:
drinks = ["water", "tea", "coffe", "wine", "espresso","oil", "vinegar", "yogurt", "cappuccino", "caffeine", "milk", "juice", "drink", "boisson", "beer"]

foundDrink = []
for id, category in category_names_dict.items():
    categoryIsDrink = False
    for drink in drinks:
        if drink in category:
            categoryIsDrink = True
    if categoryIsDrink:
        foundDrink.append(category)

print(", ".join(foundDrink))
print(len(foundDrink))

remove = [ "water", "coffee", "tea", "ristretto_with_caffeine", "juice_apple", "yogurt", "wine_red", "wine_white", 
           "white_coffee", "ice_tea_on_black_tea_basis", "beer", "glucose_drink_50g", "water_with_lemon_juice",
           "espresso", "juice_orange", "cappuccino", "juice", "chocolate_milk", "kefir_drink", "mixed_milk_beverage", "soft_drink_with_a_taste",
           "oil_vinegar_salad_dressing", "wine_rose", "cola_based_drink", "watermelon_fresh"]

remove = [''] # dont remove classes

filtered_category_names_dict = {}
for id, category in category_names_dict.items():
    if category not in remove:
        filtered_category_names_dict[id] = category

print(", ".join(filtered_category_names_dict.values()))
print(len(filtered_category_names_dict))

beetroot-steamed-without-addition-of-salt, water, coffee, tea, ristretto_with_caffeine, juice_apple, yogurt, wine_red, wine_white, green_bean_steamed_without_addition_of_salt, white_coffee, ice_tea_on_black_tea_basis, beer, glucose_drink_50g, water_with_lemon_juice, espresso, juice_orange, mashed_potatoes_prepared_with_full_fat_milk_with_butter, chicken_curry_cream_coconut_milk_curry_spices_paste, cappuccino, porridge_prepared_with_partially_skimmed_milk, juice, chocolate_milk, kefir_drink, mixed_milk_beverage, soft_drink_with_a_taste, white_bread_with_butter_eggs_and_milk, tuna_in_oil_drained, oil_vinegar_salad_dressing, wine_rose, cola_based_drink, watermelon_fresh
32
beetroot-steamed-without-addition-of-salt, bread_wholemeal, jam, water, bread, banana, soft_cheese, ham_raw, hard_cheese, cottage_cheese, coffee, fruit_mixed, pancake, tea, salmon_smoked, avocado, spring_onion_scallion, ristretto_with_caffeine, ham_n_s, egg, bacon, chips_french_fries, juice_apple, chicken, tomato, brocc

In [9]:
filtered_category_names = list(filtered_category_names_dict.values())
catIds = coco_dataset.getCatIds(catNms=filtered_category_names) #specify which categories you want to retrieve
# catIds = coco_dataset.getCatIds() #get all categories

In [20]:
# generate classesIds for the ground truth mask

classesId = {name: i+1 for i, name in enumerate(filtered_category_names_dict.values())}
filtered_category_names

['beetroot-steamed-without-addition-of-salt',
 'bread_wholemeal',
 'jam',
 'water',
 'bread',
 'banana',
 'soft_cheese',
 'ham_raw',
 'hard_cheese',
 'cottage_cheese',
 'coffee',
 'fruit_mixed',
 'pancake',
 'tea',
 'salmon_smoked',
 'avocado',
 'spring_onion_scallion',
 'ristretto_with_caffeine',
 'ham_n_s',
 'egg',
 'bacon',
 'chips_french_fries',
 'juice_apple',
 'chicken',
 'tomato',
 'broccoli',
 'shrimp_prawn',
 'carrot',
 'chickpeas',
 'french_salad_dressing',
 'pasta_hornli_ch',
 'sauce_cream',
 'pasta_n_s',
 'tomato_sauce',
 'cheese_n_s',
 'pear',
 'cashew_nut',
 'almonds',
 'lentil_n_s',
 'mixed_vegetables',
 'peanut_butter',
 'apple',
 'blueberries',
 'cucumber',
 'yogurt',
 'butter',
 'mayonnaise',
 'soup',
 'wine_red',
 'wine_white',
 'green_bean_steamed_without_addition_of_salt',
 'sausage',
 'pizza_margherita_baked',
 'salami_ch',
 'mushroom',
 'tart_n_s',
 'rice',
 'white_coffee',
 'sunflower_seeds',
 'bell_pepper_red_raw',
 'zucchini',
 'asparagus',
 'tartar_sauce',
 '

In [11]:
# Getting all categories with respective to their total images

no_images_per_category = {}
allImageIds = []
for n, i in enumerate(catIds):
  imgIds = coco_dataset.getImgIds(catIds=i)
  label = filtered_category_names[n]
  no_images_per_category[label] = len(imgIds)
  allImageIds += imgIds

allImageIds = list(set(allImageIds)) # remove duplicates
print(len(allImageIds))

no_images_per_category

54392


{'alfa_sprouts': 59,
 'almonds': 369,
 'aperitif_with_alcohol_n_s_aperol_spritz': 54,
 'apple': 1270,
 'apple_pie': 151,
 'applesauce': 109,
 'apricots': 201,
 'artichoke': 74,
 'asparagus': 216,
 'aubergine_eggplant': 300,
 'avocado': 647,
 'bacon': 426,
 'baked_potato': 256,
 'balsamic_salad_dressing': 266,
 'banana': 875,
 'banana_cake': 52,
 'basil': 65,
 'beans_kidney': 118,
 'beef_n_s': 572,
 'beer': 344,
 'beetroot-steamed-without-addition-of-salt': 281,
 'beetroot_raw': 90,
 'bell_pepper_red_raw': 806,
 'bell_pepper_red_stewed': 84,
 'berries_n_s': 127,
 'birchermuesli_prepared_no_sugar_added': 201,
 'biscuit': 287,
 'black_olives': 336,
 'blue_mould_cheese': 102,
 'blueberries': 404,
 'bolognaise_sauce': 180,
 'bouillon': 123,
 'braided_white_loaf_ch': 487,
 'bread': 1287,
 'bread_black': 117,
 'bread_fruit': 97,
 'bread_grain': 292,
 'bread_pita': 114,
 'bread_rye': 88,
 'bread_ticino_ch': 62,
 'bread_toast': 206,
 'bread_white': 2526,
 'bread_whole_wheat': 782,
 'bread_whole

In [12]:
import functools
import cv2
import numpy as np

def isNoisyAnnotation(masks):
  
  #### check if masks have high iou between each other
  intersectionThreshold = 0.3
  for mask1 in masks: 
    for mask2 in masks: # compare every mask with each other 
      if not np.array_equal(mask1,mask2): 
        intersectionPixels = np.sum(np.logical_and(mask1, mask2))
        smallestMask = min(np.sum(mask1),np.sum(mask2))
        percentualIntersection = intersectionPixels / smallestMask
        # print(percentualIntersection)
        if percentualIntersection > intersectionThreshold:
          print("Masks with high intersection")
          return True

  #### check if exists multiple contours in one mask and if these regions are too small
  areaThreshold = 0.01
  maxContours = 1

  for mask in masks:    
    mask = mask * 255 # transform binary mask from (0,1) format to (0,255)

    # fig = plt.figure(figsize=(9,9))
    # ax1 = fig.add_subplot(1,2,1)
    # ax1.imshow(mask)
    # plt.show()

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    numContours = len(contours)
  
    for contour in contours:
      area = cv2.contourArea(contour)
      if area == 0.0:
        print("Found zero area")
        numContours -= 1
        continue

      totalPixels = mask.shape[0] * mask.shape[1]
      # print(area/totalPixels)
      if area/totalPixels < areaThreshold:
        print("Very Small contour")
        return True

    # print(numContours)
    if numContours == 0:
      print("No contours")
      return True    

    if numContours > maxContours:
      print("Multiple contours")
      return True

  return False

def concatMasks(mask1, mask2):
  bothNotZero = np.logical_and(mask1, mask2)
  mask2[bothNotZero] = 0
  return mask1+mask2

def concatenateAllMasks(masks, ids, checkNoisyAnnotation=True):
    if checkNoisyAnnotation and isNoisyAnnotation(masks):
      return None

    masksWithLabel = []
    for mask, id in zip(masks, ids): 
        masksWithLabel.append(mask*id)
        
    concatenatedMasks = functools.reduce(concatMasks, masksWithLabel)
    return concatenatedMasks

In [14]:
import os

start = 0
for idx, _ in enumerate(allImageIds):
    idx = idx + start
    img_coco = coco_dataset.loadImgs(ids=allImageIds[idx])[0]
    img_path = coco_dataset_folder + "images/" + img_coco['file_name']

    annIds = coco_dataset.getAnnIds(imgIds=img_coco['id'])
    anns = coco_dataset.loadAnns(annIds)

    try:
        classes = []
        masks = []
        for ann in anns:
            mask = coco_dataset.annToMask(ann)
            masks.append(mask)
            classes.append(filtered_category_names_dict[ann["category_id"]])

    except:
        print("Skipping image that has removed class")
        continue

    maskIds = [classesId[className] for className in classes]
    # print(", ".join(classes))
    # print(maskIds)
    
    # fig = plt.figure(figsize=(9,9))
    # img = Image.open(img_path)
    # arr = np.asarray(img)
    # ax1 = fig.add_subplot(1,2,1)
    # ax1.imshow(arr)
    # coco_dataset.showAnns(anns) # optional

    finalMask = concatenateAllMasks(masks,maskIds,checkNoisyAnnotation=False)
    if finalMask is None:
        print("Skipping noisy image")
        # plt.show()
        # print("-------------------------------------")
        continue

    # ax1 = fig.add_subplot(1,2,2)
    # ax1.imshow(finalMask)
    # plt.show()
    # print("-------------------------------------")

    try:
      os.renames(img_path, coco_dataset_folder + "done/" + img_coco['file_name']) # move original file
    except:
      print("Image was already moved")
      continue
      
    im = Image.fromarray(finalMask)
    filename = img_coco['file_name'].split(".")[0]
    im.save(coco_dataset_folder + "gt/" + filename + "GT.png") # save mask




Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already moved
Image was already mov